# 🔧 도구 호출 에이전트(Tool Calling Agent)

## 📚 개요

**도구 호출 에이전트** 는 AI 모델이 필요에 따라 외부 도구를 자동으로 선택하고 실행할 수 있도록 해주는 강력한 시스템입니다. 마치 **만능 비서** 가 상황에 맞는 도구를 찾아 업무를 처리하는 것과 같습니다.

### 🔍 도구 호출이란?

도구 호출을 사용하면 모델이 하나 이상의 **도구(tool)** 가 **호출되어야 하는 시기를 감지하고 해당 도구에 전달해야 하는 입력** 으로 전달할 수 있습니다. 

### 🎯 기존 방식 vs 도구 호출 방식

#### ❌ **기존 방식 (일반 텍스트)**
```
사용자: "오늘 날씨 어때?"
AI: "죄송합니다. 실시간 날씨 정보에 접근할 수 없습니다."
```

#### ✅ **도구 호출 방식**
```
사용자: "오늘 날씨 어때?"
AI → 날씨 API 호출 → "서울은 현재 20도, 맑음입니다."
```

### 🏢 실생활 비유로 이해하기

**도구 호출 에이전트** 를 **회사의 프로젝트 매니저** 에 비유해보세요:

- **📋 요청 파악**: 고객의 요청사항을 정확히 분석
- **🔧 적절한 팀 선택**: 상황에 맞는 전문팀(도구) 배정  
- **📞 업무 지시**: 팀에게 구체적인 작업 내용 전달
- **📊 결과 취합**: 각 팀의 결과를 종합하여 최종 보고

### ⚡ 주요 장점

✅ **다양한 LLM 지원**: OpenAI 외에도 `Anthropic`, `Google Gemini`, `Mistral` 등 지원  
✅ **안정적인 호출**: 일반 텍스트보다 더 정확한 도구 호출 구현  
✅ **반복적 실행**: 문제가 해결될 때까지 여러 도구를 순차적으로 활용  
✅ **구조화된 출력**: JSON 형태의 명확한 도구 호출 결과

### 🔄 에이전트 실행 흐름

```
👤 사용자 질문 → 🤖 AI 분석 → 🔧 도구 선택 → ⚙️ 도구 실행 → 📊 결과 반환 → 🎯 최종 답변
```

**참고 링크**
- [LangChain 공식 도큐먼트](https://python.langchain.com/v0.1/docs/modules/agents/agent_types/tool_calling/)

![](./assets/agent-concept.png)

In [ ]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv(override=True)

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LangGraph-Tutorial")

In [ ]:
from langchain.tools import tool
from typing import List, Dict, Annotated
from langchain_teddynote.tools import GoogleNews
from langchain_experimental.utilities import PythonREPL


# 뉴스 검색 도구 생성
@tool
def search_news(query: str) -> List[Dict[str, str]]:
    """Search Google News by input keyword"""
    # GoogleNews 인스턴스 생성
    news_tool = GoogleNews()
    # 키워드로 뉴스 검색 (최대 5개)
    return news_tool.search_by_keyword(query, k=5)


# Python 코드 실행 도구 생성
@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    result = ""
    try:
        # PythonREPL로 코드 실행
        result = PythonREPL().run(code)
    except BaseException as e:
        # 오류 발생 시 에러 메시지 출력
        print(f"Failed to execute. Error: {repr(e)}")
    finally:
        # 실행 결과 반환
        return result


# 도구 정보 출력
print(f"도구 이름: {search_news.name}")
print(f"도구 설명: {search_news.description}")
print(f"도구 이름: {python_repl_tool.name}")
print(f"도구 설명: {python_repl_tool.description}")

In [ ]:
# 도구들을 리스트로 정의 (에이전트가 사용할 도구 목록)
tools = [search_news, python_repl_tool]

## 📝 Agent 프롬프트 생성 - AI의 지시서 만들기

**프롬프트** 는 AI 에이전트에게 주는 **업무 지시서** 와 같습니다. 마치 **신입사원에게 업무 매뉴얼** 을 주는 것처럼, AI가 어떤 역할을 하고 어떤 도구를 사용해야 하는지 명확히 알려줍니다.

### 🎭 프롬프트의 3가지 핵심 변수

#### 1️⃣ **chat_history** - 💭 과거 대화 기록
- **역할**: 이전 대화 내용을 저장하는 변수
- **비유**: 고객상담사의 **통화 이력 메모**
- **활용**: 멀티턴 대화를 지원하지 않는다면 생략 가능

#### 2️⃣ **agent_scratchpad** - 📝 임시 작업 메모장  
- **역할**: 에이전트가 현재 작업 중인 내용을 임시로 저장
- **비유**: 직장인의 **포스트잇과 메모장**
- **활용**: 도구 호출 과정과 중간 결과들을 기록

#### 3️⃣ **input** - 📥 현재 사용자 요청
- **역할**: 지금 처리해야 할 사용자의 질문이나 요청
- **비유**: 고객이 **방금 전화로 요청한 내용**
- **활용**: 매번 새롭게 입력되는 사용자 메시지

### 🏗️ 프롬프트 구조 설계

```
📋 시스템 메시지 (업무 매뉴얼)
├── 🎯 에이전트의 역할 정의
├── 🔧 사용 가능한 도구 설명  
└── 📐 작업 수행 방식 안내

💭 대화 이력 (이전 대화들)

📥 현재 입력 (지금 처리할 요청)

📝 작업 메모 (진행 중인 내용들)
```

### 💡 효과적인 프롬프트 작성 팁

✅ **명확한 역할 정의**: "당신은 뉴스 검색 전문가입니다"  
✅ **도구 사용 지침**: "뉴스 검색은 반드시 search_news 도구를 사용하세요"  
✅ **응답 스타일 지정**: "정중하고 친근한 어조로 답변하세요"  
✅ **예외 상황 대응**: "정보가 없을 때는 솔직히 모른다고 답변하세요"

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# 프롬프트 생성
# 프롬프트는 에이전트에게 모델이 수행할 작업을 설명하는 텍스트를 제공합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `search_news` tool for searching keyword related news.",
        ),
        ("placeholder", "{chat_history}"),  # 이전 대화 내용이 들어갈 자리
        ("human", "{input}"),  # 사용자 입력이 들어갈 자리
        ("placeholder", "{agent_scratchpad}"),  # 에이전트 작업 메모가 들어갈 자리
    ]
)

## 🤖 Agent 생성 - 똑똑한 AI 조립하기

이제 **LLM**, **도구들**, **프롬프트** 를 결합하여 실제로 작동하는 에이전트를 만들어 봅시다! 마치 **레고 블록을 조립** 하듯이 각 구성요소를 하나로 연결하는 과정입니다.

### 🧩 에이전트 구성 요소

- **🧠 LLM (Language Model)**: AI의 두뇌 역할
- **🔧 Tools**: 외부 작업을 수행하는 도구들
- **📋 Prompt**: AI에게 주는 업무 지시서

### 💡 create_tool_calling_agent의 역할

`create_tool_calling_agent` 함수는 이 세 가지 요소를 **유기적으로 결합** 하여 도구를 스마트하게 사용할 수 있는 에이전트를 생성합니다.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent

# LLM 정의 (OpenAI GPT-4.1 모델 사용)
llm = ChatOpenAI(model="gpt-4.1", temperature=0)

# Agent 생성 (LLM, 도구, 프롬프트를 결합)
agent = create_tool_calling_agent(llm, tools, prompt)

## 🚀 AgentExecutor - 에이전트 실행 엔진

**AgentExecutor** 는 도구를 사용하는 에이전트를 실행하는 핵심 클래스입니다. 마치 **자동차의 엔진** 처럼 에이전트가 실제로 작동할 수 있도록 해주는 실행 환경이라고 생각하시면 됩니다.

### 🎛️ 주요 구성 속성

#### 🔧 **기본 설정**
- **`agent`**: 실행할 에이전트 객체 (실제 두뇌 역할)
- **`tools`**: 에이전트가 사용할 수 있는 도구 목록
- **`return_intermediate_steps`**: 중간 단계 결과도 함께 반환할지 선택

#### ⏱️ **실행 제어**
- **`max_iterations`**: 최대 실행 단계 수 (무한루프 방지)
- **`max_execution_time`**: 최대 실행 시간 (시간 제한)
- **`early_stopping_method`**: 조기 종료 방법
  - **"force"**: 강제 중지 시 종료 메시지 반환
  - **"generate"**: 강제 중지 시 LLM이 마지막 답변 생성

#### 🛠️ **고급 기능**
- **`handle_parsing_errors`**: 출력 파싱 오류 처리 방식
  - **True**: 자동으로 오류 처리
  - **False**: 오류 발생 시 중단
  - **함수**: 커스텀 오류 처리 로직
- **`trim_intermediate_steps`**: 중간 단계 메모리 관리
  - **-1**: 모든 단계 보존
  - **함수**: 커스텀 트리밍 로직

### 🎯 주요 실행 메서드

#### 1️⃣ **invoke()** - 일반 실행
```python
result = agent_executor.invoke({"input": "질문"})
# 최종 결과만 반환
```

#### 2️⃣ **stream()** - 스트리밍 실행  
```python
for step in agent_executor.stream({"input": "질문"}):
    print(step)  # 단계별 진행 과정 확인
```

### 🏭 실생활 비유: 프로젝트 관리 시스템

**AgentExecutor** 를 **회사의 프로젝트 관리 시스템** 에 비유해보세요:

- **📊 진행 상황 모니터링**: 각 단계별 작업 진행도 추적
- **⏰ 데드라인 관리**: 최대 실행 시간으로 프로젝트 기한 관리  
- **🚨 예외 상황 대응**: 오류 발생 시 적절한 대응 방안 실행
- **📝 보고서 작성**: 중간 결과와 최종 결과를 정리해서 보고

### 💡 최적화 팁

✅ **적절한 제한 설정**: max_iterations와 max_execution_time을 상황에 맞게 조절  
✅ **메모리 관리**: 복잡한 작업에서는 trim_intermediate_steps 활용  
✅ **단계별 모니터링**: 복잡한 작업의 경우 stream() 메서드로 진행 상황 확인  
✅ **오류 처리**: handle_parsing_errors=True로 안정적인 실행 보장

In [ ]:
from langchain.agents import AgentExecutor

# AgentExecutor 생성 (에이전트 실행을 담당하는 엔진)
agent_executor = AgentExecutor(
    agent=agent,  # 실행할 에이전트
    tools=tools,  # 사용할 도구 목록
    verbose=True,  # 실행 과정 상세 출력
    max_iterations=10,  # 최대 실행 단계 수
    max_execution_time=10,  # 최대 실행 시간(초)
    handle_parsing_errors=True,  # 파싱 오류 자동 처리
)

# AgentExecutor 실행
result = agent_executor.invoke({"input": "AI 투자와 관련된 뉴스를 검색해 주세요."})

print("Agent 실행 결과:")
print(result["output"])

## 🎬 Stream 출력으로 단계별 결과 확인

**스트리밍 모드** 는 에이전트가 작업하는 과정을 실시간으로 지켜볼 수 있는 기능입니다. 마치 **요리사가 요리하는 과정** 을 실시간으로 보는 것처럼, AI가 어떤 도구를 사용하고 어떤 결과를 얻는지 단계별로 확인할 수 있습니다.

### 📺 스트리밍의 장점

✅ **투명성**: AI가 어떤 과정을 거쳐 답변에 도달했는지 명확히 확인  
✅ **디버깅**: 문제가 발생한 지점을 정확히 파악  
✅ **사용자 경험**: 긴 작업 시간 동안 진행 상황을 보여주어 안심감 제공  
✅ **학습**: AI의 문제 해결 과정을 관찰하여 인사이트 획득

### 🔄 스트리밍 출력 패턴

AgentExecutor의 `stream()` 메소드는 다음과 같은 패턴으로 출력됩니다:

```
1️⃣ Action 출력 (도구 호출)
   ↓
2️⃣ Observation 출력 (도구 실행 결과)  
   ↓
3️⃣ Action 출력 (다음 도구 호출)
   ↓  
4️⃣ Observation 출력 (다음 도구 실행 결과)
   ↓
... (목표 달성까지 반복) ...
   ↓
🎯 Final Answer (최종 답변)
```

### 📊 출력 내용 상세 분석

| 출력 타입 | 포함 내용 | 의미 |
|----------|-----------|------|
| **Action** | `actions`: 실행할 작업<br>`messages`: 도구 호출 메시지 | "지금 이 도구를 사용할 것입니다" |
| **Observation** | `steps`: 지금까지의 작업 기록<br>`messages`: 도구 실행 결과 | "도구 실행이 완료되었고 이런 결과를 얻었습니다" |
| **Final Answer** | `output`: 최종 완료 결과<br>`messages`: 사용자에게 전달할 답변 | "모든 작업이 끝났고 최종 답변은 이것입니다" |

이제 실제 예제를 통해 스트리밍이 어떻게 작동하는지 확인해봅시다! 🚀

In [ ]:
from langchain.agents import AgentExecutor

# AgentExecutor 생성 (스트리밍용 - verbose=False로 설정)
agent_executor = AgentExecutor(
    agent=agent,  # 실행할 에이전트
    tools=tools,  # 사용할 도구 목록
    verbose=False,  # 기본 출력 비활성화 (커스텀 출력 사용)
    handle_parsing_errors=True,  # 파싱 오류 자동 처리
)

In [ ]:
# 스트리밍 모드로 실행합니다
result = agent_executor.stream({"input": "AI 투자와 관련된 뉴스를 검색해 주세요."})

# 각 스트리밍 단계를 반복하면서 출력
for step in result:
    # 중간 단계 출력
    print(step)
    print("===" * 20)  # 구분선

### 🎨 중간 단계 출력을 사용자 정의 함수로 출력

기본 스트리밍 출력은 **원시 데이터** 형태로 나와서 보기에 복잡할 수 있습니다. 이를 **더 깔끔하고 이해하기 쉽게** 만들기 위해 **콜백 함수** 를 사용할 수 있습니다.

### 🔧 세 가지 핵심 콜백 함수

#### 1️⃣ **tool_callback** - 🛠️ 도구 호출 알림
- **역할**: "지금 어떤 도구를 사용하고 있는지" 알려주는 함수
- **비유**: 요리사가 "지금 오븐을 사용하겠습니다" 라고 말하는 것

#### 2️⃣ **observation_callback** - 👀 관찰 결과 처리  
- **역할**: "도구 실행 결과가 어떻게 나왔는지" 보여주는 함수
- **비유**: 요리사가 "오븐에서 빵이 잘 구워졌습니다" 라고 보고하는 것

#### 3️⃣ **result_callback** - 🎯 최종 답변 출력
- **역할**: "모든 작업이 완료되고 최종 결과" 를 정리해서 보여주는 함수  
- **비유**: 요리사가 "요리가 완성되었습니다. 맛있게 드세요!" 라고 말하는 것

### 💡 콜백 함수 사용의 장점

✅ **가독성 향상**: 복잡한 원시 데이터를 보기 좋게 포맷팅  
✅ **사용자 경험**: Streamlit 등 웹 앱에서 실시간 진행 상황 표시  
✅ **디버깅 용이**: 필요한 정보만 선별적으로 출력  
✅ **커스터마이징**: 프로젝트 요구사항에 맞는 출력 형태 구현

### 🎯 Agent 스트림 파서 - 똑똑한 출력 정리도구

**AgentStreamParser** 는 에이전트의 복잡한 중간 과정을 **보기 좋게 정리해주는 도구** 입니다. 마치 **회의록 작성 전문가** 가 복잡한 회의 내용을 핵심만 추려서 정리해주는 것과 같습니다.

### 🏢 실무 활용 시나리오

이 콜백 함수는 특히 **Streamlit** 웹 애플리케이션에서 사용자에게 에이전트의 진행 상황을 실시간으로 보여줄 때 매우 유용합니다.

#### 💼 **실무 적용 예시**
- **고객 상담 시스템**: "AI가 지금 데이터베이스를 검색하고 있습니다..."
- **데이터 분석 도구**: "차트 생성을 위해 Python 코드를 실행 중입니다..."  
- **문서 요약 서비스**: "PDF 파일에서 핵심 내용을 추출하고 있습니다..."

### 🎨 사용자 경험 개선

✅ **진행 상황 표시**: 사용자가 기다리는 동안 무엇이 진행되는지 명확히 표시  
✅ **전문적인 느낌**: 원시 출력 대신 정제된 메시지로 신뢰감 증대  
✅ **디버깅 지원**: 문제 발생 시 어느 단계에서 멈췄는지 쉽게 파악  
✅ **교육적 가치**: 에이전트가 어떻게 작동하는지 학습할 수 있음

In [ ]:
from langchain_teddynote.messages import AgentStreamParser

# Agent 스트림 파서 생성 (출력 형태를 깔끔하게 정리)
agent_stream_parser = AgentStreamParser()

### 🎪 스트리밍 방식으로 Agent 실행 과정 관찰하기

이제 실제로 에이전트가 작업하는 모습을 **실시간으로** 관찰해봅시다! 마치 **마술사의 마술 과정** 을 무대 뒤에서 지켜보는 것처럼 흥미진진한 경험이 될 것입니다.

### 🔍 관찰 포인트

다음 실행에서 다음과 같은 과정을 확인할 수 있습니다:

1. **🤖 AI 분석**: 사용자 요청을 분석하고 적절한 도구 선택
2. **🔧 도구 호출**: Python 코드 실행 도구 활용  
3. **⚠️ 오류 처리**: matplotlib 모듈이 없을 때의 대응
4. **🎯 대안 제시**: 실행 불가시 코드 예제로 대안 제공

### 💡 실행 결과 예상

matplotlib이 설치되지 않은 환경에서는 오류가 발생하지만, 에이전트가 **똑똑하게 대응** 하여 사용자에게 유용한 정보를 제공할 것입니다.

In [ ]:
# 파이차트 생성 요청으로 스트리밍 실행
result = agent_executor.stream(
    {"input": "matplotlib 을 사용하여 pie 차트를 그리는 코드를 작성하고 실행하세요."}
)

# 각 단계를 파서로 처리하여 깔끔하게 출력
for step in result:
    # 중간 단계를 parser를 사용하여 단계별로 출력
    # print(step)  # 원시 출력 (주석 처리)
    agent_stream_parser.process_agent_steps(step)

### 🛠️ 커스텀 콜백으로 출력 형태 개인화하기

이제 **나만의 스타일로** 에이전트 출력을 꾸며봅시다! 마치 **TV 뉴스 앵커** 처럼 정보를 전달하거나, **개발자 콘솔** 처럼 기술적으로 표시하는 등 원하는 형태로 커스터마이징할 수 있습니다.

### 🎨 콜백 함수 커스터마이징 예시

아래 예제에서는 각 단계를 **특별한 구분선** 으로 감싸서 더욱 눈에 띄게 만들었습니다:

- **`<<<<<< 도구 호출 >>>>>>`**: 어떤 도구가 실행되는지 알려주는 섹션  
- **`<<<<<< 관찰 내용 >>>>>>`**: 도구 실행 결과를 보여주는 섹션
- **`<<<<<< 최종 답변 >>>>>>`**: 모든 작업 완료 후 최종 결과 섹션

### 💼 실무 적용 아이디어

#### 🏢 **기업용 대시보드**
```
[INFO] 데이터베이스 연결 중...
[SUCCESS] 고객 정보 조회 완료  
[RESULT] 총 1,234건의 레코드 발견
```

#### 🎯 **사용자 친화적 인터페이스**
```
🔍 검색 중: AI 관련 뉴스를 찾고 있어요
📄 결과: 5개의 최신 뉴스를 찾았어요!
✨ 완료: 요약된 뉴스를 준비했습니다
```

### 💡 콜백 활용 팁

✅ **일관된 스타일**: 모든 출력에서 동일한 형태의 구분자나 아이콘 사용  
✅ **정보 계층화**: 중요도에 따라 다른 색상이나 스타일 적용  
✅ **진행 상황 표시**: 퍼센티지나 프로그레스 바로 진행도 시각화  
✅ **오류 친화적**: 오류 발생 시에도 사용자가 이해하기 쉬운 메시지 제공

In [ ]:
# AgentCallbacks와 AgentStreamParser를 langchain_teddynote.messages에서 가져옵니다.
from langchain_teddynote.messages import AgentCallbacks, AgentStreamParser


# 도구 호출 시 실행되는 콜백 함수입니다.
def tool_callback(tool) -> None:
    print("<<<<<<< 도구 호출 >>>>>>")
    print(f"Tool: {tool.get('tool')}")  # 사용된 도구의 이름을 출력합니다.
    print("<<<<<<< 도구 호출 >>>>>>")


# 관찰 결과를 출력하는 콜백 함수입니다.
def observation_callback(observation) -> None:
    print("<<<<<<< 관찰 내용 >>>>>>")
    print(
        f"Observation: {observation.get('observation')[0]}"
    )  # 관찰 내용을 출력합니다.
    print("<<<<<<< 관찰 내용 >>>>>>")


# 최종 결과를 출력하는 콜백 함수입니다.
def result_callback(result: str) -> None:
    print("<<<<<<< 최종 답변 >>>>>>")
    print(result)  # 최종 답변을 출력합니다.
    print("<<<<<<< 최종 답변 >>>>>>")


# AgentCallbacks 객체를 생성하여 각 단계별 콜백 함수를 설정합니다.
agent_callbacks = AgentCallbacks(
    tool_callback=tool_callback,  # 도구 호출시 콜백
    observation_callback=observation_callback,  # 관찰시 콜백
    result_callback=result_callback,  # 최종 결과시 콜백
)

# AgentStreamParser 객체를 생성하여 에이전트의 실행 과정을 파싱합니다.
agent_stream_parser = AgentStreamParser(agent_callbacks)

### 🎉 커스터마이징 결과 확인

아래의 실행 결과를 확인해보면, 이전의 복잡한 원시 출력과는 달리 **우리가 정의한 콜백 함수** 의 깔끔한 출력으로 변경된 것을 확인할 수 있습니다.

### 🔄 변화 비교

#### ❌ **이전 (원시 출력)**
```
{'agent': {'messages': [...복잡한 데이터...]}}
{'steps': [...내부 구조 데이터...]}
```

#### ✅ **현재 (커스텀 출력)**  
```
<<<<<< 도구 호출 >>>>>>
Tool: search_news
<<<<<< 도구 호출 >>>>>>

<<<<<< 관찰 내용 >>>>>>
Observation: {...뉴스 검색 결과...}
<<<<<< 관찰 내용 >>>>>>
```

### 💡 사용자 경험 개선 효과

✅ **가독성 ↑**: 복잡한 데이터 구조가 사라지고 핵심 정보만 표시  
✅ **이해도 ↑**: 각 단계가 무엇을 하는지 명확하게 구분  
✅ **시각적 효과**: 구분선으로 각 섹션을 명확히 분리  
✅ **실무 적용성**: 실제 서비스에서 사용자에게 보여줄 수 있는 형태

In [ ]:
# 뉴스 검색 요청으로 스트리밍 실행 (커스텀 콜백 적용)
result = agent_executor.stream({"input": "AI 투자관련 뉴스를 검색해 주세요."})

# 각 단계를 커스텀 파서로 처리하여 출력
for step in result:
    # 중간 단계를 parser 를 사용하여 단계별로 출력
    agent_stream_parser.process_agent_steps(step)

## 🧠 이전 대화내용 기억하는 Agent - 기억력을 가진 AI

지금까지의 에이전트는 **금붕어** 처럼 매번 대화를 새로 시작했습니다. 이제 **인간처럼 기억력** 을 가진 에이전트를 만들어보겠습니다!

### 🤖 기존 에이전트의 한계

```
👤 사용자: "안녕, 내 이름은 홍길동이야"  
🤖 AI: "안녕하세요!"

👤 사용자: "내 이름이 뭐였지?"
🤖 AI: "죄송합니다. 이름을 알려주시지 않으셨어요." ❌
```

### 🧠 기억력을 가진 에이전트

```
👤 사용자: "안녕, 내 이름은 홍길동이야"
🤖 AI: "안녕하세요, 홍길동님!"

👤 사용자: "내 이름이 뭐였지?"  
🤖 AI: "홍길동님이라고 하셨어요!" ✅
```

### 🔧 RunnableWithMessageHistory 활용

**RunnableWithMessageHistory** 는 에이전트에게 **기억 장치** 를 달아주는 도구입니다. 마치 사람의 **해마(기억을 담당하는 뇌 부위)** 와 같은 역할을 합니다.

### 🏢 실생활 비유: 고객 상담 시스템

**기억력 있는 에이전트** 를 **전문 고객 상담사** 에 비유해보세요:

#### 📋 **상담 이력 관리**
- **고객 정보**: 이름, 이전 문의 내역, 선호사항 기억
- **진행 상황**: 이전 상담에서 어디까지 진행했는지 파악  
- **개인화 서비스**: 고객의 특성에 맞는 맞춤 응답

#### 🗂️ **세션별 관리**  
- **session_id**: 각 고객별로 별도의 상담 기록 유지
- **대화 연결성**: "지난번에 말씀하신 그 건은..." 형태의 자연스러운 대화

### 💡 메모리 기능의 장점

✅ **자연스러운 대화**: 매번 자기소개를 반복할 필요 없음  
✅ **문맥 유지**: 이전 대화의 맥락을 이어가며 더 정확한 답변  
✅ **개인화**: 사용자의 선호도나 이전 요청사항을 기억하여 맞춤 서비스  
✅ **효율성**: 이미 제공한 정보를 다시 요청하지 않아도 됨

**참고 링크**
- [RunnableWithMessageHistory 상세 가이드](https://wikidocs.net/254682)

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# session_id를 저장할 딕셔너리 생성 (각 사용자별 대화 기록 저장소)
store = {}


# session_id를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    if session_ids not in store:  # session_id가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# 채팅 메시지 기록이 추가된 에이전트를 생성합니다.
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,  # 기존 에이전트 실행기
    # 대화 session_id별 히스토리 관리 함수
    get_session_history,
    # 프롬프트의 질문이 입력되는 key: "input"
    input_messages_key="input",
    # 프롬프트의 메시지가 입력되는 key: "chat_history"
    history_messages_key="chat_history",
)

In [ ]:
# 첫 번째 대화 - 자기소개 (abc123 세션에서)
response = agent_with_chat_history.stream(
    {"input": "안녕? 내 이름은 테디야!"},
    # session_id 설정 (각 사용자별로 다른 ID 사용)
    config={"configurable": {"session_id": "abc123"}},
)

# 출력 확인 (파서를 통해 깔끔하게 출력)
for step in response:
    agent_stream_parser.process_agent_steps(step)

In [ ]:
# 두 번째 대화 - 이름 기억 테스트 (같은 abc123 세션)
response = agent_with_chat_history.stream(
    {"input": "내 이름이 뭐라고?"},
    # 같은 session_id 사용 (이전 대화 기억)
    config={"configurable": {"session_id": "abc123"}},
)

# 출력 확인 (AI가 이전에 말한 이름을 기억하는지 확인)
for step in response:
    agent_stream_parser.process_agent_steps(step)

In [ ]:
# 세 번째 대화 - 추가 개인 정보 제공 (abc123 세션 계속)
response = agent_with_chat_history.stream(
    {
        "input": "내 이메일 주소는 teddy@teddynote.com 이야. 회사 이름은 테디노트 주식회사야."
    },
    # 같은 session_id 사용 (대화 맥락 유지)
    config={"configurable": {"session_id": "abc123"}},
)

# 출력 확인
for step in response:
    agent_stream_parser.process_agent_steps(step)

In [ ]:
# 네 번째 대화 - 복잡한 업무 요청 (이전 정보 활용)
response = agent_with_chat_history.stream(
    {
        "input": "최신 뉴스 5개를 검색해서 이메일의 본문으로 작성해줘. "
        "수신인에는 `셜리 상무님` 그리고, 발신인에는 내 인적정보를 적어줘."
        "정중한 어조로 작성하고, 메일의 시작과 끝에는 적절한 인사말과 맺음말을 적어줘."
    },
    # 같은 session_id 사용 (이전 대화에서 언급한 개인정보 활용)
    config={"configurable": {"session_id": "abc123"}},
)

# 출력 확인 (AI가 이전 대화의 이름, 이메일, 회사정보를 기억해서 활용하는지 확인)
for step in response:
    agent_stream_parser.process_agent_steps(step)